# Capstone Project - The Battle of the Neighborhoods (Final Assignment)

*This notebook is mainly for Applied Data Science Capstone - Coursera Course, and should not be used for references or clinical judgement*

## Table of contents
- [Introduction](#introduction)
- [Data](#data)
- [Methodology and Analysis](#methodology-and-analysis)
- [Results](#results)
- [Discussion](#discussion)
- [Conclusion](#conclusion)


## Introduction

Coronavirus disease (COVID-19) is an infectious disease caused by a newly discovered coronavirus.  
In Thailand, the city which has most cases of COVID-19 is Bangkok - the capital and most populous city of Thailand.
The city divided into 50 districts in total.  
In this project we will try to find the correlation of number of COVID-19 cases who had onset in each district of Bangkok and the clusters of districts based on catogery and density of venues

This data and visualization may provided useful information for strategy to deal with current COVID-19 spreads.
For example may promotes the awareness of people, or lead to invented policies to limited the spread of the disease.

## Data

The data we need are list below  

- Number of COVID-19 cases who had onset in each district of Bangkok - this data is query and process from https://covid19.th-stat.com/api/open/cases - official API provided by Department of Disease Control of Thailand
- Coordinates data of each district of Bangkok - together with Foursquare API to find the venues and create clusters of district. I used data from https://data.go.th/dataset/item_c6d42e1b-3219-47e1-b6b7-dfe914f27910 - Open Government Data of Thailand. Which included the data of latitude and longitude of each subdistrict in Thailand
- Table included lists of district in Bangkok included both English and Thai name of each district from https://en.wikipedia.org/wiki/List_of_districts_of_Bangkok
- GeoJSON file of districts in Bangkok from https://github.com/pcrete/gsvloader-demo/blob/master/geojson/Bangkok-districts.geojson for visualize chloropleth map

### COVID-19 cases

### Coordinates data

### Foursquare

## Methodology and Analysis

## Results

## Discussion

## Conclusion